In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [10]:
os.chdir('/home/julien/Dropbox/Python')
ls

NameError: name 'ls' is not defined

In [11]:
ls

corr_matrix.tif               machineleraning.ipynb  Untitled3.ipynb
cross_corr.ipynb              meta_analysis/         Untitled.ipynb
cross_corr.xlsx               ML_Workshop/           wowah_ML_workshop.ipynb
HarmAversion_USV_Dec2USV.csv  tomtom/                Xebia_Python/
HarmAversion_USV_Total.csv    Untitled1.ipynb
HarmAversion_USV_USV2Dec.csv  Untitled2.ipynb


In [2]:
# Set path
# Ubuntu
#work_dir = "/home/julien/Documents/meta_analysis"
# Windows
work_dir = "C:\\Users\hernandez\Dropbox (Personal)\Python\meta_analysis"
os.chdir(work_dir)
# Verify the path
cwd = os.getcwd() 
# Print the current directory 
print("Current working directory is:", cwd) 

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hernandez\\Dropbox (Personal)\\Python\\meta_analysis'

In [3]:
files = os.listdir(cwd)
imported_dat = pd.read_excel(io         = files[files.index("Effect_size_pooled_c-fos_v2.xlsx")], 
                             sheet_name ='c-fos_only',
                             header     = 1,
                             index_col  = None)

# lower case to avoid later conflicts
imported_dat.columns = [c.lower() for c in imported_dat.columns]
# display columns
#imported_dat.columns

In [4]:
# Create dictionnary of the species encoded
specie_dict ={0 : 'rat', 1 : 'mouse'}
# Map the binary coding onto the dictionnary
imported_dat['specie_str'] = imported_dat['specie '].map(specie_dict)
# Select data of interest
imported_dat = imported_dat[['study id','specie ', 'specie_str', 'structure / tissue', 'structure_renamed',  
                             'standardized es',    'standardized es se', 'weight factor', 'n']]

In [5]:
(imported_dat
.groupby(['specie_str'])
.apply(lambda x: (x.groupby('structure_renamed')
.count()
.sort_values('n', ascending=False))
.head(50))
)

study id  specie   specie_str  \
specie_str structure_renamed                                  
mouse      Amygd_BLA                 4        4           4   
           HPC                       3        3           3   
           Amygd_LA                  3        3           3   
           PrL                       2        2           2   
           Amygd_CeL                 2        2           2   
           Amygd_CeM                 2        2           2   
           Amygd_CoA                 2        2           2   
           Amygd_MeA                 2        2           2   
           IL                        2        2           2   
           NaCC                      1        1           1   
           PVN                       1        1           1   
           PFC                       1        1           1   
           PAG                       1        1           1   
           ACC                       1        1           1   
           INS                       1        1           1   
           Hypothalamus              1        1           1   
           BNST                      1        1           1   
           Amygd_CeA                 1        1           1   
           Septum                    1        1           1   
rat        Amygd_LA                 14       14          14   
           Amygd_CeA                 8        8           8   
           PAG                       8        8           8   
           PVN                       7        7           7   
           IL                        7        7           7   
           PrL                       6        6           6   
           Amygd_BLA                 6        6           6   
           Amygd_BA                  6        6           6   
           Hypothalamus              6        6           6   
           ACC                       5        5           5   
           Amygd_CeM                 5        5           5   
           Amygd_CoA                 5        5           5   
           Amygd_MeA                 5        5           5   
           NaCC                      4        4           4   
           BMA                       3        3           3   
           INS                       3        3           3   
           BNST                      3        3           3   
           Amygd_CeL                 3        3           3   
           PFC                       3        3           3   
           HPC                       2        2           2   
           Septum                    2        2           2   
           LPO                       1        1           1   

                              structure / tissue  standardized es  \
specie_str structure_renamed                                        
mouse      Amygd_BLA                           4                4   
           HPC                                 3                3   
           Amygd_LA                            3                3   
           PrL                                 2                2   
           Amygd_CeL                           2                2   
           Amygd_CeM                           2                2   
           Amygd_CoA                           2                2   
           Amygd_MeA                           2                2   
           IL                                  2                2   
           NaCC                                1                1   
           PVN                                 1                1   
           PFC                                 1                1   
           PAG                                 1                1   
           ACC                                 1                1   
           INS                                 1                1   
           Hypothalamus                        1                1   
           BNST                                1                1   
         

In [6]:
(imported_dat
#.drop_duplicates(subset =["study id"], inplace = False)
.groupby(['specie_str'])
.count()
)

,study id,specie,structure / tissue,structure_renamed,standardized es,standardized es se,weight factor,n
specie_str,,,,,,,,
mouse,32,32,32,32,32,32,32,32
rat,112,112,112,112,112,112,112,112


In [7]:
# This function will add a few new variables to the dataset
def compute_meta_data(dataf):
    return (dataf
            .assign(vi_SE2           = lambda d:  1/d['weight factor'])
            .assign(standardized_ESU = lambda d:  d['standardized es'] * (1-(3/(4*(d['n'])-9)))) 
            .assign(wES              = lambda d:  d['standardized es'] * d['weight factor'])
            .assign(wES2             = lambda d:  d['weight factor']   * d['standardized es'] * d['standardized es']) 
            .assign(w2               = lambda d:  d['weight factor']   * d['weight factor']))


# This function will compute, for each specie&structure, the random effect parameters
def compute_random_effects(dataf):
    return (dataf
            .groupby(['specie_str','structure_renamed'])
            .apply(lambda d: pd.Series({  
            "sum_wi":          d['weight factor'].sum(),
            "sum_wi2":         d['w2'].sum(),
            "k":               d['w2'].count(), #num studies
            "wxES":            d['wES'].sum(),
            "wxES2":           d['wES2'].sum()     
            })))

def compute_random_variance(dataf):
    return (dataf
            .assign(Q  = lambda d:  abs(d['wxES2']-(d['wxES']**2/d['sum_wi']))) 
            # For struture with only 1 effect size, the denominator of v0 will automatically
            # be equal to 0. hence v0 will not be computed.
            .assign(v0 = lambda d:  (d['Q']-(d['k']-1))/(d['sum_wi']-
                                                        (d['sum_wi2']/d['sum_wi'])))
           )

def zero_if_negative(dataf):
        num = dataf['v0']
        num[num < 0] = 0
        dataf['v0'] = num
        return dataf

def apply_corrections(dataf):
    return (dataf
            .assign(v0_plus_vi       = lambda d:  d['vi_SE2']+d['v0'])  
            .assign(wi_corr          = lambda d:  1/d['v0_plus_vi'])  
            .assign(wxES_corr        = lambda d:  d['wi_corr']*d['standardized_ESU'])  
            .assign(wxESsq_corr      = lambda d:  d['wi_corr']*(d['standardized_ESU']**2))
            .assign(SE_corr          = lambda d:  (1/(d['wi_corr'])**0.5))  
            .fillna(0) # NaN are replaced by 0
            .replace(np.inf, 0)
)

def calculate_constants(dataf):
    return (dataf
            .groupby(['specie_str', 'structure_renamed'])
            .apply(lambda d: pd.Series({  
                        "sum_wxES_corr":        d['wxES_corr'].sum(),
                        "sum_wxESsq_corr":      d['wxESsq_corr'].sum(), #num studies                  
                        "sum_wi_corr":          abs(d['wi_corr']).sum(), 
             })) 
             .fillna(0) # NaN are replaced by 0
             .replace(np.inf, 0)
           )

# Computes the I^2 value, ie, percent of variation due to heterogeneity rather than chance
# By convention, Q = 0 if Q < k-1, so that the precision of a random effects summary estimate 
# will not exceed the precision of a fixed effect summary estimate
# See Higgins & Thompson 2002; DOI: 10.1002/sim.1186
def I2(Q,dfg):
#     if pd.isnull((Q < dfg)):
#         I2 = 0
#         return I2
#     else:
        I2=((Q-dfg)/Q)*100
        return I2

def calculate_mean_se(dataf):
    return (dataf
            .assign(ES_mean       = lambda d: d['sum_wxES_corr']/d['sum_wi_corr']) 
            .assign(SE_mean       = lambda d: (1/d['sum_wi_corr'])**0.5)
            .assign(z             = lambda d: d['ES_mean']/d['SE_mean'])
            .assign(high_CI       = lambda d: d['ES_mean']+(1.96*d['SE_mean']))
            .assign(low_CI        = lambda d: d['ES_mean']-(1.96*d['SE_mean']))                                              
           # .assign(k_val         = lambda d: d['w2'].count())#num studies  
            .assign(I_val         = lambda d: I2(d['Q'],d['k']-1))
           )

def calculate_number_es(dataf):
    return (dataf
            .groupby(['specie_str', 'structure_renamed'])
            .apply(lambda d: pd.Series({  
                        "k_val": d['w2'].count(),
             })) 
           )

In [40]:
# Compute & add meta variables to dataset
cfos_df = imported_dat.pipe(compute_meta_data)
# Compute & add random effect parameters
rand_df = (cfos_df
.pipe(compute_random_effects)
.pipe(compute_random_variance)
.pipe(zero_if_negative)
)
# Merge datasets
cfos_ana = pd.merge(cfos_df, rand_df, on=['structure_renamed','specie_str'])
cfos_ana = apply_corrections(cfos_ana)
# Compute constants for each structure
constants = calculate_constants(cfos_ana)
cfos_ana = pd.merge(cfos_ana, constants, on=['structure_renamed','specie_str'])
# Compute mean & se for each structure
cfos_ana = (cfos_ana
.pipe(calculate_mean_se))
# Calculate number of ES per structure
nES = (cfos_ana
.pipe(calculate_number_es)           
)
cfos_ana = pd.merge(cfos_ana, nES, on=['structure_renamed','specie_str'])

# Sample dataframe
cfos_ana
#cfos_ana[['Q', 'k_val', 'I_val']].head(30)
#cfos_ana[cfos_ana['structure / tissue'].str.contains("NaCC") ]
#temp = cfos_ana[['specie_str', 'structure_renamed','ES_mean', 'low_CI','high_CI', 'z', 'I_val', 'k_val']]
#temp.groupby(["specie_str"]).apply(lambda x: x.sort_values(["ES_mean"], ascending = False)).reset_index(drop=True).head(30)

,study id,specie,specie_str,structure / tissue,structure_renamed,standardized es,standardized es se,weight factor,n,vi_SE2,...,sum_wxES_corr,sum_wxESsq_corr,sum_wi_corr,ES_mean,SE_mean,z,high_CI,low_CI,I_val,k_val
0,2,0,rat,ACC,ACC,0.927872,0.235702,18,21,0.055556,...,18.885442,15.12257,32.517006,0.580787,0.175366,3.311859,0.924503,0.23707,40.641371,5
1,2,0,rat,ACC,ACC,0.678397,0.258199,15,18,0.066667,...,18.885442,15.12257,32.517006,0.580787,0.175366,3.311859,0.924503,0.23707,40.641371,5
2,207,0,rat,ACC,ACC,0.744927,0.242536,17,20,0.058824,...,18.885442,15.12257,32.517006,0.580787,0.175366,3.311859,0.924503,0.23707,40.641371,5
3,53,0,rat,ACC,ACC,-0.333586,0.447214,5,8,0.200000,...,18.885442,15.12257,32.517006,0.580787,0.175366,3.311859,0.924503,0.23707,40.641371,5
4,53,0,rat,ACC,ACC,0.382893,0.447214,5,8,0.200000,...,18.885442,15.12257,32.517006,0.580787,0.175366,3.311859,0.924503,0.23707,40.641371,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,259,1,mouse,BNST,BNST,0.145535,0.377964,7,10,0.142857,...,0.000000,0.00000,0.000000,NaN,inf,NaN,NaN,NaN,NaN,1
140,259,1,mouse,CeA,Amygd_CeA,0.099684,0.377964,7,10,0.142857,...,0.000000,0.00000,0.000000,NaN,inf,NaN,NaN,NaN,NaN,1
141,259,1,mouse,PVN,PVN,0.646091,0.377964,7,10,0.142857,...,0.000000,0.00000,0.000000,NaN,inf,NaN,NaN,NaN,100.000000,1
142,259,1,mouse,Hypothalamus,Hypothalamus,0.388797,0.377964,7,10,0.142857,...,0.000000,0.00000,0.000000,NaN,inf,NaN,NaN,NaN,NaN,1


In [43]:
# By convention, Q = 0 if Q < k-1, so that the precision of a random effects summary estimate 
# will not exceed the precision of a fixed effect summary estimate
# See Higgins & Thompson 2002; DOI: 10.1002/sim.1186
cfos_ana.loc[cfos_ana['Q'] < (cfos_ana['k_val']-1), 'I_val'] = 0 

,study id,specie,specie_str,structure / tissue,structure_renamed,standardized es,standardized es se,weight factor,n,vi_SE2,...,sum_wxES_corr,sum_wxESsq_corr,sum_wi_corr,ES_mean,SE_mean,z,high_CI,low_CI,I_val,k_val
0,2,0,rat,ACC,ACC,0.927872,0.235702,18,21,0.055556,...,18.885442,15.122570,32.517006,0.580787,0.175366,3.311859,0.924503,0.237070,40.641371,5
1,2,0,rat,ACC,ACC,0.678397,0.258199,15,18,0.066667,...,18.885442,15.122570,32.517006,0.580787,0.175366,3.311859,0.924503,0.237070,40.641371,5
2,207,0,rat,ACC,ACC,0.744927,0.242536,17,20,0.058824,...,18.885442,15.122570,32.517006,0.580787,0.175366,3.311859,0.924503,0.237070,40.641371,5
3,53,0,rat,ACC,ACC,-0.333586,0.447214,5,8,0.200000,...,18.885442,15.122570,32.517006,0.580787,0.175366,3.311859,0.924503,0.237070,40.641371,5
4,53,0,rat,ACC,ACC,0.382893,0.447214,5,8,0.200000,...,18.885442,15.122570,32.517006,0.580787,0.175366,3.311859,0.924503,0.237070,40.641371,5
5,2,0,rat,LA,Amygd_LA,0.927872,0.235702,18,21,0.055556,...,11.454080,13.453992,32.490922,0.352532,0.175436,2.009459,0.696386,0.008677,75.981846,14
6,77,0,rat,LA,Amygd_LA,0.768121,0.447214,5,8,0.200000,...,11.454080,13.453992,32.490922,0.352532,0.175436,2.009459,0.696386,0.008677,75.981846,14
7,77,0,rat,LA,Amygd_LA,0.123303,0.447214,5,8,0.200000,...,11.454080,13.453992,32.490922,0.352532,0.175436,2.009459,0.696386,0.008677,75.981846,14
8,77,0,rat,LA,Amygd_LA,0.060945,0.447214,5,8,0.200000,...,11.454080,13.453992,32.490922,0.352532,0.175436,2.009459,0.696386,0.008677,75.981846,14
9,153,0,rat,LA,Amygd_LA,0.000000,0.267261,14,17,0.071429,...,11.454080,13.453992,32.490922,0.352532,0.175436,2.009459,0.696386,0.008677,75.981846,14


In [44]:
# Prepare data for plotting
temp = (cfos_ana
#.groupby(['specie_str'])
.drop_duplicates(subset =["specie_str", "structure_renamed"], inplace = False)
.dropna()
#.groupby(['specie_str', 'structure_renamed'])
#.head(20)
)
temp = temp[['specie_str', 'structure_renamed','ES_mean', 'low_CI','high_CI', 'z', 'I_val', 'k_val']]
# temp.columns
temp

,specie_str,structure_renamed,ES_mean,low_CI,high_CI,z,I_val,k_val
0,rat,ACC,0.580787,0.237070,0.924503,3.311859,40.641371,5
5,rat,Amygd_LA,0.352532,0.008677,0.696386,2.009459,75.981846,14
19,rat,HPC,0.235805,-0.164278,0.635889,1.155206,0.000000,2
21,rat,Amygd_CeM,-0.077602,-0.469602,0.314398,-0.388008,0.000000,5
26,rat,Amygd_CeL,0.020364,-0.485705,0.526434,0.078871,0.000000,3
29,rat,PrL,0.375268,0.072834,0.677703,2.432017,0.000000,6
35,rat,IL,0.188953,-0.051107,0.429012,1.542728,7.079613,7
42,rat,INS,-0.022349,-0.528419,0.483720,-0.086559,0.000000,3
45,mouse,PrL,0.227964,-0.570073,1.026001,0.559887,56.913815,2
47,mouse,IL,0.250577,-0.273255,0.774409,0.937572,0.000000,2


In [54]:
temp.groupby(["specie_str"]).apply(lambda x: x.sort_values(["ES_mean"], ascending = False)).reset_index(drop=True)

,specie_str,structure_renamed,ES_mean,low_CI,high_CI,z,I_val,k_val
0,mouse,Amygd_BLA,0.305320,-0.012634,0.623274,1.882120,-205.014460,4
1,mouse,IL,0.250577,-0.273255,0.774409,0.937572,-32.442447,2
2,mouse,PrL,0.227964,-0.570073,1.026001,0.559887,56.913815,2
3,mouse,Amygd_CeL,0.111359,-0.412473,0.635191,0.416668,-1071.083074,2
4,mouse,Amygd_CeM,0.045174,-0.478658,0.569006,0.169027,-1101.055235,2
5,mouse,Amygd_MeA,0.025072,-0.498760,0.548904,0.093812,-197.728568,2
6,mouse,Amygd_CoA,0.004438,-0.569050,0.577927,0.015169,16.567528,2
7,mouse,HPC,-0.131467,-0.559174,0.296240,-0.602458,-199.585540,3
8,mouse,Amygd_LA,-0.177468,-0.605175,0.250240,-0.813258,-84.773431,3
9,rat,ACC,0.580787,0.237070,0.924503,3.311859,40.641371,5


In [19]:
#import plotnine as p9
import plotnine as p9
from plotnine import ggplot, geom_point, aes, theme, element_text, save_as_pdf_pages
p9.options.figure_size = (6.4, 4.8)

In [42]:
# (p9.ggplot() + 
#  p9.geom_point(data = cfos_ana, mapping = p9.aes('specie_str', 'N_ES', color='mean_ESU')) +
#  #p9.geom_errorbar(data = c_fos_specie, mapping = p9.aes(x="specie_str", y="N_ES", ymin="std_ESU",ymax="std_ESU")) +
#  p9.facet_wrap("structure_renamed")
# )


In [48]:
char = (p9.ggplot() +
p9.geom_point(data = cfos_ana, mapping = p9.aes('structure_renamed', 'k_val', color='ES_mean', size='ES_mean')) +
p9.facet_wrap("specie_str") + 
theme(axis_text_x=element_text(rotation=90, hjust=1))
)
#save_as_pdf_pages(facet_pages('cyl'))
p9.ggplot.save(char, filename='test', format='pdf', path=None, width=None, height=None,)


C:\Users\hernandez\AppData\Local\Continuum\anaconda3\lib\site-packages\plotnine\ggplot.py:729: PlotnineWarning: Saving 6.4 x 4.8 in image.
  from_inches(height, units), units), PlotnineWarning)
C:\Users\hernandez\AppData\Local\Continuum\anaconda3\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: test
  warn('Filename: {}'.format(filename), PlotnineWarning)
C:\Users\hernandez\AppData\Local\Continuum\anaconda3\lib\site-packages\mizani\bounds.py:352: RuntimeWarning: invalid value encountered in less
  outside = (x < range[0]) | (x > range[1])
C:\Users\hernandez\AppData\Local\Continuum\anaconda3\lib\site-packages\mizani\bounds.py:352: RuntimeWarning: invalid value encountered in greater
  outside = (x < range[0]) | (x > range[1])
C:\Users\hernandez\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
C:\Users\hernandez\AppData\Local\Continuum\anaconda3\lib\site-packages\plotnine\

In [49]:
pwd

'C:\\Users\\hernandez\\Dropbox (Personal)\\Python\\meta_analysis'